In [8]:
from pprint import pprint 
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

In [9]:
from blogging.models import *  

In [4]:
Post.objects.all().first().id

1

In [5]:
def get_post_details(post_id):
    try:
        post = Post.objects.select_related('user').prefetch_related('likes').get(id=post_id)
    except Post.DoesNotExist:
        return{}
    
   # like_count = Likes.objects.filter(post=post, liked=True).count()
    like_count = post.likes.filter(liked=True).count()
    
    response_data={
        "id":post_id,
        "created_by":{
            "name": post.user.name,
            "id": post.user.id
        },
        "created_date": post.created_at,
        "likes": like_count,
        "title": post.title,
        "content": post.content
    }
    
    return response_data

In [6]:
get_post_details(post_id=1)

{'id': 1,
 'created_by': {'name': 'Palak', 'id': 1},
 'created_date': datetime.datetime(2024, 6, 11, 11, 48, 43, 533294, tzinfo=datetime.timezone.utc),
 'likes': 1,
 'title': 'Simply Styled: Effortless Fashion Tips',
 'content': "Welcome to Simply Styled, your destination for easy fashion advice! We're here to make looking great a breeze with quick tips and tricks that anyone can master. Stay tuned for effortless ways to elevate your everyday style!"}

In [153]:
from rest_framework.serializers import ModelSerializer, DateTimeField, SerializerMethodField

class UserSerializer(ModelSerializer):
    created_date = DateTimeField(source="created_at", format="%Y-%m-%d at %H:%M:%S")
    
    class Meta:
        model= User
        fields = ("id","name","created_date","likes")
                    
class PostSerializer(ModelSerializer):
    likes = SerializerMethodField()
        
    def get_likes(self, obj):
        like_count = obj.likes.count()
        return like_count
    
    created_by = UserSerializer(source="user")
    created_date = DateTimeField(source="created_at", format="%Y-%m-%d at %H:%M:%S")
    
    class Meta:    
        model = Post
        fields =("id","created_by","title","content","created_date","likes","category")
    

In [107]:
def get_post_details(post_id):
    try:
        post = Post.objects.select_related('user').prefetch_related('likes').get(id=post_id)
    except Post.DoesNotExist:
        return{}
    
    like_count = post.likes.filter(liked=True).count()
    
    post_serializer = PostSerializer(post)
    response_data= post_serializer.data
    return response_data

In [46]:
get_post_details(post_id=3)

{'id': 3, 'created_by': {'id': 3, 'name': 'Kunal'}, 'title': 'Easy Health Hacks for Busy Lives: Elevate Your Wel', 'content': "In today's hectic world, maintaining health can feel like an uphill battle. However, by integrating simple yet effective strategies into your routine, you can effortlessly boost your well-being. Start by prioritizing hydration—carry a water bottle and set reminders to drink regularly, enhancing energy and bodily functions. Incorporate movement throughout your day, opting for stairs over elevators and quick stretches during breaks to invigorate both body and mind. Plan meals ahead for healthier eating, prepping ingredients to ensure nutritious choices even on busy days.\r\n\r\nPrioritize quality sleep by establishing a calming bedtime routine and aiming for 7-9 hours nightly, rejuvenating your body and mind for optimal function. Finally, practice mindful eating—savor each bite, listen to hunger cues, and minimize distractions to foster a healthier relationship w

In [146]:
def get_user_details(user_id):
    try:
        user = User.objects.get(id=user_id)
        post = Post.objects.select_related('user').prefetch_related('likes').get(id=post_id)
    except User.DoesNotExist:
        return{}
    
   # like_count = Likes.objects.filter(post=post, liked=True).count()
    like_count = post.likes.filter(liked=True).count()
    
    response_data={
        "id":user_id,
        "name": user.name,
        "created_date": user.created_at,
        "user_stats":{
            "likes": like_count,
            "created_post": post.created_at
        }
    }
    
    return response_data

In [147]:
User.objects.all().first().id

1

In [158]:
def get_user_details(user_id, post_id):
    try:
        user = User.objects.get(id=user_id)
        post = Post.objects.select_related('user').prefetch_related('likes').get(id=post_id)
    
    except Post.DoesNotExist:
        return{}
    
    like_count = post.likes.filter(liked=True).count()
    
    
    user_serializer = UserSerializer(user)
    
    response_data=user_serializer.data    
    return response_data

In [170]:
get_user_details(user_id=1, post_id=1)

{'id': 1, 'name': 'Palak', 'created_date': '2024-06-11 at 11:44:39', 'likes': [1, 2]}

In [106]:
get_user_details(user_id=3, post_id=1)

{'id': 3,
 'name': 'Kunal',
 'created_date': datetime.datetime(2024, 6, 11, 11, 50, 15, 532225, tzinfo=datetime.timezone.utc),
 'user_stats': {'likes': 1,
  'created_post': datetime.datetime(2024, 6, 11, 11, 48, 43, 533294, tzinfo=datetime.timezone.utc)}}

In [12]:
get_user_details(user_id=2, post_id=3)

{'id': 2,
 'name': 'Tisha',
 'created_date': datetime.datetime(2024, 6, 11, 11, 49, 45, 673226, tzinfo=datetime.timezone.utc),
 'user_stats': {'likes': 1,
  'created_post': datetime.datetime(2024, 6, 11, 11, 56, 36, 996314, tzinfo=datetime.timezone.utc)}}

In [78]:
def get_recently_liked_posts(user_id):
    try:
        user = User.objects.get(id=user_id)
        #post = Post.objects.select_related('user').prefetch_related('likes').filter()
        
    except User.DoesNotExist:
        return{}
    
    liked_posts = (
        Post.objects.filter(like__user=user, like__liked=True)
        .select_related('user')
        .prefetch_related('likes')
        .order_by('-like__created_at')[:5]
    )        

    
   # like_count = Likes.objects.filter(post=post, liked=True).count()
    #like_count = post.likes.filter(liked=True).count()
    
    recently_liked_posts: List[PostDetails] = []
    for post in liked_posts:
        like_count = post.likes.filter(liked=True).count()
        post_details: PostDetails = {
        
        "id": post.id,
        "title": post.title,
        "content": post.content,
        "liked_by":{
            "name": user.name,
            "id":user_id
        },
        "created_by":{
           "name": post.user.name,
            "id": post.user.id
        },
        "created_date": post.created_at,
        "like": like_count,
        }
            
        recently_liked_posts.append(post_details)

    return recently_liked_posts


In [82]:
get_recently_liked_posts(user_id=1)

[{'id': 1,
  'title': 'Simply Styled: Effortless Fashion Tips',
  'content': "Welcome to Simply Styled, your destination for easy fashion advice! We're here to make looking great a breeze with quick tips and tricks that anyone can master. Stay tuned for effortless ways to elevate your everyday style!",
  'liked_by': {'name': 'Palak', 'id': 1},
  'created_by': {'name': 'Palak', 'id': 1},
  'created_date': datetime.datetime(2024, 6, 11, 11, 48, 43, 533294, tzinfo=datetime.timezone.utc),
  'like': 1},
 {'id': 3,
  'title': 'Easy Health Hacks for Busy Lives: Elevate Your Wel',
  'content': "In today's hectic world, maintaining health can feel like an uphill battle. However, by integrating simple yet effective strategies into your routine, you can effortlessly boost your well-being. Start by prioritizing hydration—carry a water bottle and set reminders to drink regularly, enhancing energy and bodily functions. Incorporate movement throughout your day, opting for stairs over elevators and qu

In [2]:
from rest_framework import serializers
class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = ['id', 'name']

class PostSerializer(serializers.ModelSerializer):
    created_by = UserSerializer(source='user')
    liked_by = UserSerializer(source='like__user', read_only=True)
    likes = serializers.IntegerField(source='like_count', read_only=True)
    created_date = DateTimeField(source="created_at", format="%Y-%m-%d at %H:%M:%S")

    class Meta:
        model = Post
        fields = ['id', 'title', 'content', 'created_by', 'created_date', 'liked_by', 'likes']


NameError: name 'DateTimeField' is not defined

In [472]:
def get_recently_liked_posts(user_id):
    try:
        user = User.objects.get(id=user_id)
    except User.DoesNotExist:
        return {}

    liked_posts = (
        Post.objects.filter(like__user=user, like__liked=True)
        .select_related('user')
        .prefetch_related('likes')
        .order_by('-like__created_at')[:5]
    )

    # Add the like_count to each post
    for post in liked_posts:
        post.like_count = post.likes.filter(liked=True).count()

    # Serialize the data
    serializer = PostSerializer(liked_posts, many=True, context={'user': user})
    response_data = serializer.data  
    
    return response_data


In [479]:
get_recently_liked_posts(user_id=1)

[{'id': 1, 'title': 'Simply Styled: Effortless Fashion Tips', 'content': "Welcome to Simply Styled, your destination for easy fashion advice! We're here to make looking great a breeze with quick tips and tricks that anyone can master. Stay tuned for effortless ways to elevate your everyday style!", 'created_by': {'id': 1, 'name': 'Palak'}, 'created_date': '2024-06-11 at 11:48:43', 'likes': 1}, {'id': 3, 'title': 'Easy Health Hacks for Busy Lives: Elevate Your Wel', 'content': "In today's hectic world, maintaining health can feel like an uphill battle. However, by integrating simple yet effective strategies into your routine, you can effortlessly boost your well-being. Start by prioritizing hydration—carry a water bottle and set reminders to drink regularly, enhancing energy and bodily functions. Incorporate movement throughout your day, opting for stairs over elevators and quick stretches during breaks to invigorate both body and mind. Plan meals ahead for healthier eating, prepping in

In [475]:
def get_popular_posts():

    now = timezone.now()
    seven_days_ago = now.replace(hour=0, minute=0, second=0, microsecond=0) - timezone.timedelta(days=7)

    # Fetch the top 5 posts created in the last 7 days, ordered by the count of likes
    popular_posts = (
        Post.objects.filter(created_at__gte=seven_days_ago)
        .annotate(like_count=Count('like'))
        .order_by('-like_count')[:5]
        .select_related('user')
    )

    popular_posts_data: List[PostDetails] = []
    for post in popular_posts:
        post_details: PostDetails = {
            "id": post.id,
            "title": post.title,
            "content": post.content,
            "created_by": {
                "name": post.user.name,
                "id": post.user.id
            },
            "created_date": post.created_at.isoformat(),
            "likes": post.like_count
        }
        popular_posts_data.append(post_details)

    return popular_posts_data

In [476]:
get_popular_posts()

[{'id': 4,
  'title': 'The Daily Meal',
  'content': 'The Daily Meal is a comprehensive food and drink website that was launched in 2011. It aims to cover all aspects of the culinary world, offering readers a wide array of content that ranges from recipes and cooking tips to restaurant reviews and travel guides. The blog is known for its extensive coverage of food-related topics, making it a one-stop destination for food enthusiasts.',
  'created_by': {'name': 'Palak', 'id': 1},
  'created_date': '2024-06-12T18:47:41.339732+00:00',
  'likes': 0}]

In [59]:
get_popular_posts()

[{'id': 1,
  'title': 'Simply Styled: Effortless Fashion Tips',
  'content': "Welcome to Simply Styled, your destination for easy fashion advice! We're here to make looking great a breeze with quick tips and tricks that anyone can master. Stay tuned for effortless ways to elevate your everyday style!",
  'created_by': {'name': 'Palak', 'id': 1},
  'created_date': '2024-06-11T11:48:43.533294+00:00',
  'likes': 1},
 {'id': 3,
  'title': 'Easy Health Hacks for Busy Lives: Elevate Your Wel',
  'content': "In today's hectic world, maintaining health can feel like an uphill battle. However, by integrating simple yet effective strategies into your routine, you can effortlessly boost your well-being. Start by prioritizing hydration—carry a water bottle and set reminders to drink regularly, enhancing energy and bodily functions. Incorporate movement throughout your day, opting for stairs over elevators and quick stretches during breaks to invigorate both body and mind. Plan meals ahead for heal

In [463]:
from rest_framework.serializers import ModelSerializer, DateTimeField, SerializerMethodField

class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = ['id', 'name']

class PostSerializer(serializers.ModelSerializer):
    created_by = UserSerializer(source='user')
    likes = serializers.IntegerField(source='like_count')
    created_date = DateTimeField(source="created_at", format="%Y-%m-%d at %H:%M:%S")


    class Meta:
        model = Post
        fields = ['id', 'title', 'content', 'created_by', 'created_date', 'likes']
        
class PostTypeDetailsSerializer(serializers.ModelSerializer):
    name = serializers.CharField(source='category')
    post_count = serializers.IntegerField()
    
    class Meta:
        model = Post
        fields = ['name','post_count']
        


In [445]:
def get_popular_posts():
    now = timezone.now()
    seven_days_ago = now.replace(hour=0, minute=0, second=0, microsecond=0) - timezone.timedelta(days=7)

    # Fetch the top 5 posts created in the last 7 days, ordered by the count of likes
    popular_posts = (
        Post.objects.filter(created_at__gte=seven_days_ago)
        .annotate(like_count=Count('like'))
        .order_by('-like_count')[:5]
        .select_related('user')
    )

    # Serialize the data
    serializer = PostSerializer(popular_posts, many=True)
    response_data = serializer.data

    return response_data


In [446]:
get_popular_posts()

[{'id': 4, 'title': 'The Daily Meal', 'content': 'The Daily Meal is a comprehensive food and drink website that was launched in 2011. It aims to cover all aspects of the culinary world, offering readers a wide array of content that ranges from recipes and cooking tips to restaurant reviews and travel guides. The blog is known for its extensive coverage of food-related topics, making it a one-stop destination for food enthusiasts.', 'created_by': {'id': 1, 'name': 'Palak'}, 'created_date': '2024-06-12 at 18:47:41', 'likes': 0}]

In [466]:
def get_post_type_details():
    # Annotate each post type with the count of posts
    post_type_counts = (
        Post.objects.values('category')
        .annotate(post_count=Count('id'))
        .order_by('category')
    )

    # Serialize the data
    serializer = PostTypeDetailsSerializer(post_type_counts, many=True)
    response_data = serializer.data

    return response_data


In [467]:
get_post_type_details()

[{'name': 'Fashion', 'post_count': 1}, {'name': 'Food blogs', 'post_count': 2}, {'name': 'Health and fitness blogs', 'post_count': 1}]

In [459]:
def get_post_type_details():
    
    # Annotate each post type with the count of posts
    post_type_counts = (
        Post.objects.values('category')
        .annotate(post_count=Count('id'))
        .order_by('category')
    )

    post_type_details: List[PostTypeDetails] = []
    for post in post_type_counts:
        post_type_detail: PostTypeDetails = {
            "name": post['category'],
            "post_count": post['post_count']
        }
        post_type_details.append(post_type_detail)

    return post_type_details

In [460]:
get_post_type_details()

[{'name': 'Fashion', 'post_count': 1},
 {'name': 'Food blogs', 'post_count': 2},
 {'name': 'Health and fitness blogs', 'post_count': 1}]